In [ ]:
!pip install datasets
!pip install transformers==4.2.0

In [ ]:
from datasets import load_dataset

dataset = load_dataset("medical_questions_pairs")

In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification #BertForSequenceClassification
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
tokenizer = AutoTokenizer.from_pretrained("allenai/biomed_roberta_base")

#model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
#model = BertForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = RobertaForSequenceClassification.from_pretrained("allenai/biomed_roberta_base")

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2)

In [ ]:
dataset

In [ ]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
def tokenize(batch):
    return tokenizer(batch['question_1']+"[SEP]"+batch["question_2"], padding="max_length", truncation=True, max_length=512)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=False)
test_dataset = test_dataset.map(tokenize, batched=False)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
len(train_dataset[0]["input_ids"])

In [ ]:
len(train_dataset[1]["input_ids"])

In [ ]:
train_dataset

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()